In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from skimage.io import imread
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from collections import defaultdict

In [ ]:
def random_keypoints(image, num_keypoints):
    keypoints = []
    height, width = image.shape[:2]
    
    # 计算每个网格的大小
    grid_size_x = width // int(np.sqrt(num_keypoints))
    grid_size_y = height // int(np.sqrt(num_keypoints))
    
    # 在每个网格中选择一个点作为关键点
    for i in range(0, width, grid_size_x):
        for j in range(0, height, grid_size_y):
            x = i + grid_size_x // 2
            y = j + grid_size_y // 2
            keypoints.append(cv2.KeyPoint(x, y, 10))  # 10是关键点的尺度
            
            # 如果已经选择了足够多的关键点，则停止
            if len(keypoints) == num_keypoints:
                return keypoints
    
    return keypoints

In [ ]:
def extract_features(image_path, num_keypoints):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    sift = cv2.SIFT_create()
    keypoints = random_keypoints(img, num_keypoints)
    keypoints, descriptors = sift.compute(img, keypoints)
    return descriptors

In [ ]:
def build_vocabulary(train_data_dir, num_clusters, num_keypoints):
    # 定义提取特征的函数
    def extract_features_from_folder(folder_path, num_keypoints=num_keypoints):
        features = []
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img_features = extract_features(img_path,num_keypoints=num_keypoints)
            features.extend(img_features)
        return features
    
    # 遍历每个类别文件夹并提取特征
    all_features = []
    for class_name in os.listdir(train_data_dir):
        class_dir = os.path.join(train_data_dir, class_name)
        if not os.path.isdir(class_dir):
            continue
        class_features = extract_features_from_folder(class_dir, num_keypoints=num_keypoints)
        all_features.extend(class_features)
    
    # 将提取的特征转换成特征矩阵
    features_matrix = np.array(all_features)
    
    # 应用K均值算法进行聚类
    kmeans = KMeans(n_clusters=num_clusters, n_init='auto')
    kmeans.fit(features_matrix)
    
    # 得到词典，即聚类中心
    vocabulary = kmeans.cluster_centers_
    
    return vocabulary

In [ ]:
def get_image_files_and_integer_labels(base_dir, image_extensions=['.jpg', '.jpeg', '.png', '.gif', '.bmp']):
    # Sort folder names case-insensitively and create the mapping
    folders = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))], key=lambda x: x.lower())
    folder_to_int_label = {folder.lower(): i+1 for i, folder in enumerate(folders)}  # Mapping is lowercase

    image_files = []
    int_labels = []

    for folder in folders:
        folder_path = os.path.join(base_dir, folder)
        for extension in image_extensions:
            for file in glob.glob(os.path.join(folder_path, '*' + extension)):
                image_files.append(file)
                # Assign labels using lowercase folder name for consistency
                int_labels.append(folder_to_int_label[folder.lower()])
    
    return image_files, int_labels

In [ ]:
def get_hist(image_files, int_labels, vocabulary,num_keypoints):
    histograms = []
    for image_file, label in zip(image_files, int_labels):
        img_features = extract_features(image_file,num_keypoints=num_keypoints)  # 提取图像特征
        hist = np.zeros(len(vocabulary))
        for feature in img_features:
            # 计算图像的直方图
            distances = np.linalg.norm(vocabulary - feature, axis=1)
            nearest_word_idx = np.argmin(distances)
            hist[nearest_word_idx] += 1
        histograms.append((hist, label))  # 存储直方图和对应的标签
    return histograms

In [ ]:
train_data_dir = "/users/huangsicheng/Desktop/DSA5203/Assignment3/train"

In [ ]:
def train(train_data_dir, num_clusters, num_keypoints):
    # 步骤1：构建词汇表
    vocabulary = build_vocabulary(train_data_dir, num_clusters=num_clusters,num_keypoints=num_keypoints)
    # 步骤2:获取图片文件路径并按字母顺序标签
    image_files, int_labels = get_image_files_and_integer_labels(train_data_dir)
    # 步骤3：获取视觉单词的直方图
    histograms = get_hist(image_files, int_labels, vocabulary,num_keypoints)

    # 获取每个类别的总样本数
    num_samples_per_class = len(image_files) // len(set(int_labels))

    # 创建字典以保存每个类别的图片索引
    class_indices = defaultdict(list)
    for idx, (_, label) in enumerate(histograms):
        class_indices[label].append(idx)

    # 计算应该分配给测试集的每个类别的图片数量
    num_test_samples_per_class = {label: num_samples_per_class // 20 for label in set(int_labels)}

    # 初始化训练集和测试集
    X_train, X_test, y_train, y_test = [], [], [], []

    # 从每个类别中分别选择图片添加到训练集和测试集中
    for label, indices in class_indices.items():
        # 按比例选择图片索引
        np.random.seed(42)
        test_indices = np.random.choice(indices, num_test_samples_per_class[label], replace=False)
        train_indices = [idx for idx in indices if idx not in test_indices]

        # 添加到训练集和测试集
        for idx in train_indices:
            X_train.append(histograms[idx][0])
            y_train.append(histograms[idx][1])
        for idx in test_indices:
            X_test.append(histograms[idx][0])
            y_test.append(histograms[idx][1])

    # 转换为NumPy数组
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_test, y_test = np.array(X_test), np.array(y_test)


    # 初始化高斯核支持向量机分类器
    svm_classifier = SVC(kernel='rbf')

    # 使用一对所有方法包装分类器
    ovr_classifier = OneVsRestClassifier(svm_classifier)

    # 在训练集上训练分类器
    ovr_classifier.fit(X_train, y_train)

    # 在测试集上进行预测
    y_pred = ovr_classifier.predict(X_test)

    # 计算准确率
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [ ]:
accuracy = train(train_data_dir, num_clusters=400, num_keypoints=400)